# Statistical Modeling III

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns

In the previous session, we focused on **linear regression**, starting with a **single predictor** and then extending the idea to **multiple linear regression (MLR)**. We learned how to **fit regression models using Ordinary Least Squares (OLS)**, how to **assess overall model significance**, and how to determine **which predictors meaningfully contribute** to explaining variation in the response variable.

Today, we build directly on that foundation by adding a crucial new component: **uncertainty**. While point estimates give us a single predicted value, they do not tell us how reliable that prediction is.

We will learn how to **quantify uncertainty in model predictions** using **confidence intervals** and **prediction intervals**. These tools allow us to answer important practical questions such as:

- *How confident are we about the **mean response** at a given value of the predictor?*
- *How much variability should we expect in **individual future observations**?*

Understanding the difference between these intervals is essential for interpreting regression results correctly and for making statistically sound conclusions from our models.

Before introducing these concepts, we will first fit the same **simple linear regression model** as in the previous session, using the **Palmer Penguins dataset**. In this model, **flipper length** is the independent variable (predictor), and **body mass** is the dependent variable (response).


#### <font color="#fc7202">Task 1: Fitting a simple linear regression model</font>

Let’s load the dataset and fit the model.


In [ ]:
# Load the Palmer Penguins dataset and remove observations with missing values in flipper length or body mass to ensure a clean regression analysis
penguins = sns.load_dataset('penguins').dropna(subset=['flipper_length_mm', 'body_mass_g']).reset_index(drop=True)

# Fit a simple linear regression model using Ordinary Least Squares (OLS), with body mass as the response variable and flipper length as the predicto
model1 = smf.ols('body_mass_g ~ flipper_length_mm', data=penguins).fit()

# Display a detailed summary of the fitted regression model
print(model1.summary())

#### Confidence Intervals (CIs)

So far, we have already interpreted several parts of the regression output. But there is one more *very important* piece of information in the model summary: the columns with the brackets:

`[0.025      0.975]`

The numbers in these columns give us the **confidence intervals (CIs)** for the regression coefficients reported in the summary.

> **Why is this a 95% confidence interval?**
>
> The two values $0.025$ and $0.975$ correspond to the **2.5th percentile** and the **97.5th percentile** of the sampling distribution.  
> This means we keep the middle **95%** of the distribution and leave out **2.5% in each tail**: $0.975 - 0.025 = 0.95$
>
> So the interval shown in the output is a **95% confidence interval**.

In regression, a confidence interval provides a **range of plausible values** for the *true population parameter* (for example, the true $\beta_1$), based on our sample data.

A **95% CI for $\hat{\beta}_1$** means: If we repeatedly collected new samples, fit the same regression model each time, and computed a 95% CI each time, then about **95% of those intervals** would contain the *true* $\beta_1$.

> In other words: The 95% CI reflects the **uncertainty** in our estimate. It tells us how precisely we have estimated the model’s coefficients.


> **Link to statistical significance**
>
> If the confidence interval for a coefficient **does not include $0$**, it suggests that this predictor likely has a **statistically significant effect** on the response variable (at the $5\%$ level).\
This matches the conclusion from the ***t*-test** and its ***p*-value** in the regression summary.

Next, we will extract and interpret these confidence intervals directly in Python using our fitted model `model1`.


If we want the confidence intervals for our fitted model, we can use the model’s `.conf_int(alpha=...)` method, where  $\alpha = 1 - \text{confidence level}$.

In [ ]:
model1.conf_int(alpha=0.05) # 95% CIs (same as shown in .summary())

We are not restricted to a 95% confidence level. We can choose different confidence levels depending on the context and how much uncertainty we are willing to tolerate.

For example, a 90% confidence interval corresponds to $\alpha = 0.10$:


In [ ]:
model1.conf_int(alpha=0.10)  # 90% CIs

Lower confidence levels produce **narrower intervals**, while higher confidence levels produce **wider intervals**, reflecting a trade-off between precision and certainty.

##### Visualizing the confidence bands

**Confidence bands** show how the **uncertainty of the estimated mean response** $\hat{y}(x)$ changes across different values of $x$.  
In other words, they illustrate how confident we are about the **average predicted value** of $y$ at each point along the regression line.

We can compute and plot these confidence bands in Python using the model’s `.get_prediction()` and `.summary_frame()` methods.

> The method `.get_prediction()` in `statsmodels` allows us to compute **predicted values** along with the associated **uncertainty estimates** for a regression model.  
> By default, if no new data are provided, it returns predictions for all observations used to fit the model.


In [ ]:
pred_ols = model1.get_prediction()
sf = pred_ols.summary_frame(alpha=0.05)
sf

> - The method `.get_prediction()` creates a prediction-results object that contains not only the fitted values $\hat{y}_i$, but also all the necessary information to compute **standard errors** and **confidence or prediction intervals**.  
>   The optional argument `exog=` can be used to specify new $x$ values.  
>   Since we did not specify `exog`, it returns predictions for all data points used when fitting the model.
>
> - The `.summary_frame()` method then converts these prediction results into a **DataFrame** containing all the relevant quantities for each observation, including the fitted value, its standard error, and the bounds of both **confidence** and **prediction intervals**.


In [ ]:
# Prepare data for plotting
# Sort by flipper_length_mm so lines/bands render correctly
order = np.argsort(penguins['flipper_length_mm'].values)
x_sorted = penguins['flipper_length_mm'].values[order]
y_sorted = penguins['body_mass_g'].values[order]
mean_sorted = sf['mean'].values[order]
lo_sorted = sf['mean_ci_lower'].values[order]
hi_sorted = sf['mean_ci_upper'].values[order]

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], s=15, alpha=0.6, label='observed data', color='darkgray')
ax.plot(x_sorted, mean_sorted, lw=2, color='#004aad', label='fitted line (mean)')
ax.fill_between(x_sorted, lo_sorted, hi_sorted, alpha=0.2, color='#004aad', label='95% CI (mean)')
ax.set_xlabel('Flipper length (mm)')
ax.set_ylabel('Body mass (g)')
ax.legend(frameon=False)
plt.tight_layout()
plt.show()


> As an alternative, we can create a very similar visualization using `seaborn` and its `regplot` function.  
> This provides a convenient way to plot the **regression line together with its 95% confidence interval** using a single command.


In [ ]:
# Creating the scatterplot with regression line using seaborn
plt.figure(figsize=(5, 5))

sns.regplot(data=penguins, x='flipper_length_mm', y='body_mass_g', color='darkgray',
    ci=95,  # plot 95% confidence interval for the mean response
    scatter_kws={'s': 15, 'alpha': 0.6},
    line_kws={'color': '#004aad', 'lw': 2}
)

plt.xlabel('Flipper length (mm)')
plt.ylabel('Body mass (g)')
plt.tight_layout()
plt.show()


As you may notice, the **confidence bands are curved**. They are **narrowest near the center** of the data and **widen symmetrically toward both ends**. This reflects increasing uncertainty as we move away from the mean of $x$ (in this case, the mean value of flipper length).

This curvature does **not** imply that the model allows for a curved relationship. The regression model itself is still **strictly linear**.

The curved shape of the confidence bands represents the **envelope of all plausible straight lines** that could result from random sampling variability in the estimated intercept and slope.

> To see *why* this happens, we can use a simple simulation approach. We will repeatedly:
> - randomly sample 50 penguins from the dataset,
> - fit the same simple linear regression model to each sample, and
> - repeat this process many times (for example, 1000 times).
>
> Overlaying these fitted lines will show how uncertainty in the estimated coefficients leads to greater spread at the edges of the data range, producing the characteristic curved shape of the confidence bands.


#### <font color="#fc7202">Task 2: Simulation of sampling variability in linear regression</font>

To understand how this simulation works, we will start with a **single simulation step**.  
We randomly select **50 penguins** from the dataset, fit a **simple linear regression model**, and visualize the fitted line together with the data.


In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Randomly sample 50 penguins (with replacement)
sample = penguins.sample(n=50, replace=True)
sample

In [ ]:
# Fit the simple linear regression model
model_sim = smf.ols('body_mass_g ~ flipper_length_mm', data=sample).fit()

# Create a grid of x values for plotting the fitted line
x_grid = np.linspace(penguins['flipper_length_mm'].min(), penguins['flipper_length_mm'].max(), 100)

# Predict body mass over the x grid
y_pred = model_sim.predict({'flipper_length_mm': x_grid})

# Plot original data (light gray)
plt.figure(figsize=(5, 5))
plt.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], alpha=0.3, color='lightgray', label='Original data')


# Plot sampled data (highlighted)
plt.scatter(sample['flipper_length_mm'],sample['body_mass_g'], alpha=0.8, color='#fc7202', label='Sample (n = 50)')

# Plot fitted regression line
plt.plot(x_grid, y_pred, color='#fc7202', label='Fitted regression line')

# Labels and legend
plt.xlabel('Flipper length (mm)')
plt.ylabel('Body mass (g)')
plt.legend()

plt.show()

Now let’s take the next step.

We will **draw a new random sample**, fit the same **simple linear regression model**, and **plot it on the same graph as the previous one**.  
> Each new sample and its fitted line will be shown in **different colors**, while the original data remain unchanged in the background.


In [ ]:
# Set random seed for reproducibility
np.random.seed(2)

# New random sample of 50 penguins
sample2 = penguins.sample(n=50, replace=True)

# Fit the simple linear regression model
model_sim2 = smf.ols('body_mass_g ~ flipper_length_mm', data=sample2).fit()

# Create a grid of x values for plotting the fitted line
x_grid = np.linspace(penguins['flipper_length_mm'].min(), penguins['flipper_length_mm'].max(), 100)

# Predict body mass over the x grid
y_pred2 = model_sim2.predict({'flipper_length_mm': x_grid})

# Plot original data (light gray)
plt.figure(figsize=(5, 5))
plt.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], alpha=0.3, color='lightgray', label='Original data')

# Plot first sample and fitted line
plt.scatter(sample['flipper_length_mm'],sample['body_mass_g'], alpha=0.8, color='#fc7202', label='Sample 1 (n = 50)')
plt.plot(x_grid, y_pred, color='#fc7202', label='Fitted regression line 1')

# Plot second sample and fitted line
plt.scatter(sample2['flipper_length_mm'],sample2['body_mass_g'], alpha=0.8, color='#004aad', label='Sample 2 (n = 50)')
plt.plot(x_grid, y_pred2, color='#004aad', label='Fitted regression line 2')

# Labels and legend
plt.xlabel('Flipper length (mm)')
plt.ylabel('Body mass (g)')
plt.legend()

plt.show()



By comparing these fitted lines, we can directly see how the regression model varies from sample to sample due to **random sampling variability**, even though the assumed relationship is the same.

Now let’s repeat this procedure **1000 times** and overlay all fitted regression lines on the same plot.

In [ ]:
# Simulation to illustrate why confidence bands are curved
# Set random seed for reproducibility
np.random.seed(42)

# Number of simulations and sample size
n_simulations = 1000
sample_size = 50

# Create a grid of x values for plotting the fitted lines
x_grid = np.linspace(penguins['flipper_length_mm'].min(), penguins['flipper_length_mm'].max(), 100)

# Plot the original data (light gray for context)
plt.figure(figsize=(5, 5))
plt.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], color='lightgray', alpha=0.3)

# Run the simulation
for simulation in range(n_simulations):
    # Randomly sample 50 penguins
    sample = penguins.sample(n=sample_size, replace=True)
    
    # Fit the same simple linear regression model
    model_sim = smf.ols('body_mass_g ~ flipper_length_mm', data=sample).fit()
    
    # Predict body mass over the x grid
    y_pred = model_sim.predict({'flipper_length_mm': x_grid})
    
    # Plot the fitted line (very transparent)
    plt.plot(x_grid, y_pred, color='#004aad')

# Plot labels
plt.xlabel('Flipper length (mm)')
plt.ylabel('Body mass (g)')
plt.show()


As you can see, even though every fitted line is straight, together they fill out a **curved envelope**.

This shape arises because:
- Near $\bar{x}$, the fitted lines are tightly clustered, indicating **small uncertainty**.  
- Far from $\bar{x}$, the fitted lines diverge in both directions, indicating **larger uncertainty**.



Under the usual linear regression assumptions, we can say that we are **95% confident** that the true best-fit line lies within these curved boundaries.  
However, remember that these **confidence bands refer to the mean response**, not individual data points. Many observations will fall outside the band, as you can also see here.  
If we wanted to capture most of the *data points* instead, we would use a **prediction interval**, which is considerably wider.

##### Visualizing the prediction bands

**Prediction bands** show how the **uncertainty for an individual outcome** $Y$ changes across values of $x$.  

Unlike **confidence bands**, which describe uncertainty in the **mean response** $\hat{y}(x)$, prediction bands answer a different question:

> *“If $X = x$, where might the outcome of a **new individual** fall?”*

**Interpretation at a fixed $x$:**  
A **95% prediction interval** at $X = x$ means that, for individuals in the population with this same value of $x$, about **95% of their outcomes** are expected to fall **within the interval**.

Another way to think about it is this:  
If we repeatedly observe **new individuals** with the same $x$, their values of $Y$ would lie inside the prediction interval **about 95% of the time**.

Prediction bands are **wider than confidence bands** because they account for **two sources of uncertainty**:
1. Uncertainty in the **estimated mean response** $\hat{y}(x)$.  
2. The **natural variability between individuals**, represented by the error variance $\sigma^2$.

We can compute and plot **prediction bands** in Python using the same `.get_prediction()` and `.summary_frame()` output, but this time using the columns `obs_ci_lower` and `obs_ci_upper`.


In [ ]:
# Getting model predictions and prediction intervals for each observation
pred_ols = model1.get_prediction()
sf = pred_ols.summary_frame(alpha=0.05)

# Sorting by x to ensure the lines and bands are plotted correctly
order = np.argsort(penguins['flipper_length_mm'].values)
x_sorted = penguins['flipper_length_mm'].values[order]
y_sorted = penguins['body_mass_g'].values[order]
mean_sorted = sf['mean'].values[order]
pi_low_sorted = sf['obs_ci_lower'].values[order]
pi_high_sorted = sf['obs_ci_upper'].values[order]

# Plotting the prediction band
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], s=15, alpha=0.6, label='observed data', color='darkgray')
ax.plot(x_sorted, mean_sorted, lw=2, color='#570a6d', label='fitted line (mean)')
ax.fill_between(x_sorted, pi_low_sorted, pi_high_sorted, alpha=0.2, color='#570a6d', label='95% prediction band')

ax.set_xlabel('Flipper length (mm)')
ax.set_ylabel('Body mass (g)')
ax.legend()
plt.tight_layout()
plt.show()


> We can also visualize both bands on the same plot.

In [ ]:
# Getting model predictions, confidence intervals, and prediction intervals
pred_ols = model1.get_prediction()
sf = pred_ols.summary_frame(alpha=0.05)

# Sort by flipper_length_mm so lines and bands render correctly
order = np.argsort(penguins['flipper_length_mm'].values)
x_sorted = penguins['flipper_length_mm'].values[order]
y_sorted = penguins['body_mass_g'].values[order]
mean_sorted = sf['mean'].values[order]
ci_low_sorted = sf['mean_ci_lower'].values[order]
ci_high_sorted = sf['mean_ci_upper'].values[order]
pi_low_sorted = sf['obs_ci_lower'].values[order]
pi_high_sorted = sf['obs_ci_upper'].values[order]

# Plot confidence and prediction bands together
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], s=15, alpha=0.6, label='observed data', color='darkgray')
ax.plot(x_sorted, mean_sorted, lw=2, color='#004aad', label='fitted line (mean)')
ax.fill_between(x_sorted, ci_low_sorted, ci_high_sorted, alpha=0.25, color='#004aad', label='95% CI (mean)')
ax.fill_between(x_sorted, pi_low_sorted, pi_high_sorted, alpha=0.18, color='#570a6d', label='95% prediction band')
ax.set_xlabel('Flipper length (mm)')
ax.set_ylabel('Body mass (g)')
ax.legend(frameon=False)
plt.tight_layout()
plt.show()


#### <font color="#fc7202">Task 3: Modeling toxicity using chemical properties</font>

In this task, we will put the theory into practice.

Our goal is to develop a statistical model to predict **chemical toxicity** for fathead minnows. Toxicity is measured as **$\text{LC}_{50}$**, which represents the concentration of a chemical that causes **50% mortality**.

Based on scientific knowledge from environmental chemistry and ecotoxicology, we know that **chemical hydrophobicity** plays a key role in toxicity. A common measure of hydrophobicity is the **octanol/water partition coefficient** ($K_\text{OW}$), which is often used in its logarithmic form, **$\log P$**. More hydrophobic chemicals tend to bioaccumulate more easily and may therefore be more toxic to aquatic organisms.

To do this, you are provided with a file `toxicity.tsv`, which contains measurements for multiple chemicals:
- octanol/water partition coefficient ($K_\text{OW}$)
- $\text{LC}_{50}$ values for fathead minnows

In the following steps, we will use this dataset to explore the relationship between chemical hydrophobicity and toxicity, check the underlying statistical assumptions, and build an appropriate regression model.

---

**Step 1: Exploring the relationship between variables**

First, we investigate whether there is a **linear relationship** between toxicity ($\text{LC}_{50}$) and hydrophobicity ($K_\text{OW}$), as suggested by theory.

To do this, we will:
- **Visually inspect** the relationship using an appropriate plot.
- Compute the **Pearson correlation coefficient** to quantify the strength and direction of the linear association.

> Keep in mind that the Pearson correlation is a **parametric measure**, which relies on certain assumptions.





In [ ]:
# Load the toxicity dataset
toxicity = pd.read_csv('toxicity.tsv', sep='\t')

In [ ]:
# Create a scatter plot to examine the relationship between the variables,
# and histograms to check whether they are approximately normally distributed,
# which is an assumption of Pearson’s correlation coefficient
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

# Scatter plot
sns.scatterplot(data=toxicity, x='Kow', y='LC50 (mmol/L)', s=25, color='#fc7202', ax=axes[0])
axes[0].set_xlabel('Kow')
axes[0].set_ylabel('LC50 (mmol/L)')
axes[0].set_title('Relationship between Kow and LC50')

# Histograms
sns.histplot(toxicity['Kow'], bins=20, color='darkgray', ax=axes[1])
axes[1].set_xlabel('Kow')
axes[1].set_title('Distribution of Kow')

sns.histplot(toxicity['LC50 (mmol/L)'], bins=20, color='darkgray', ax=axes[2])
axes[2].set_xlabel('LC50 (mmol/L)')
axes[2].set_title('Distribution of LC50')

plt.tight_layout()
plt.show()

> Based on the plots, we can see that **neither variable is normally distributed**. Both **$K_\text{OW}$** and **$\text{LC}_{50}$** show strong right skewness, with most observations concentrated at low values and a few very large values.\
> This pattern is consistent with what we would expect for **concentration-related variables**, which often follow a **log-normal distribution** rather than a normal one.\
> Because the assumption of normality is violated, using the raw variables is not ideal for Pearson correlation or linear regression. A common and scientifically meaningful solution is to apply a **log-transformation** to both variables.\
> Therefore, in the next step, we will **log-transform $K_\text{OW}$ and $\text{LC}_{50}$** and then repeat the visualizations to check whether the transformed variables are closer to being normally distributed and whether the relationship between


In [ ]:
# Log-transform the variables to achieve approximate normality
toxicity['logP'] = np.log10(toxicity['Kow'])
toxicity['log_LC50'] = np.log10(toxicity['LC50 (mmol/L)'])

toxicity.head()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

# Scatter plot
sns.scatterplot(data=toxicity, x='logP', y='log_LC50', s=25, color='#fc7202', ax=axes[0])
axes[0].set_xlabel('logP')
axes[0].set_ylabel('log(LC50)')
axes[0].set_title('Relationship between logP and log(LC50)')

# Histograms
sns.histplot(toxicity['logP'], bins=20, color='darkgray', ax=axes[1])
axes[1].set_xlabel('logP')
axes[1].set_title('Distribution of logP')

sns.histplot(toxicity['log_LC50'], bins=20, color='darkgray', ax=axes[2])
axes[2].set_xlabel('log(LC50)')
axes[2].set_title('Distribution of log(LC50)')

plt.tight_layout()
plt.show()

> After applying the logarithmic transformation, we can see a clear improvement in the distributions of both variables. The heavy right skew observed in the original scale is no longer present, and the data are much more symmetrically distributed.\
> The scatterplot of **$\log P$** versus **$\log (\text{LC}_{50})$** also suggests a more linear relationship, which is more consistent with the assumptions of Pearson correlation and linear regression.\
> However, from histograms alone it is still **visually difficult to determine** whether the transformed variables are approximately normally distributed. To assess normality more formally, we will therefore also examine **Q–Q plots**, which allow us to compare the empirical distributions of the transformed variables to a theoretical normal distribution.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))

# Q-Q plot for logP
res1 = stats.probplot(toxicity['logP'], dist='norm', plot=axes[0])
axes[0].get_lines()[0].set_markerfacecolor('#fc7202')
axes[0].get_lines()[0].set_markeredgecolor('#fc7202')
axes[0].get_lines()[1].set_color('black')
axes[0].get_lines()[1].set_linestyle('--')
axes[0].set_title('Q-Q Plot: logP', fontsize=11)
axes[0].set_xlabel('Theoretical quantiles')
axes[0].set_ylabel('Sample quantiles')

# Q-Q plot for log(LC50)
res2 = stats.probplot(toxicity['log_LC50'], dist='norm', plot=axes[1])
axes[1].get_lines()[0].set_markerfacecolor('#fc7202')
axes[1].get_lines()[0].set_markeredgecolor('#fc7202')
axes[1].get_lines()[1].set_color('black')
axes[1].get_lines()[1].set_linestyle('--')
axes[1].set_title('Q-Q Plot: log(LC50)', fontsize=11)
axes[1].set_xlabel('Theoretical quantiles')
axes[1].set_ylabel('Sample quantiles')

plt.tight_layout()
plt.show()

> Based on the Q–Q plots, the **log-transformed variables align closely with the 45° reference line**, indicating that the data are **approximately normally distributed on the log scale**.\
> Of course, to formally confirm normality we would need to apply dedicated statistical tests, such as the **Shapiro–Wilk** or **Kolmogorov–Smirnov** tests. However, for the purposes of this analysis, we proceed under the reasonable assumption that the variables are now **approximately normally distributed**.
>
> This allows us to **compute the Pearson correlation coefficient** and to test whether the relationship between **$\log P$** and **$\log (\text{LC}_{50})$** is **statistically significant**.


In [ ]:
pearson_r, pearson_p = pearsonr(toxicity['logP'], toxicity['log_LC50'])

print(f"Correlation (logP vs log(LC50)): r = {pearson_r:.3f}, p = {pearson_p:.4f}")

> Based on these results, we observe a **strong negative linear relationship** between **$\log P$** and **$\log (\text{LC}_{50})$**, with a Pearson correlation coefficient of $r = -0.818$.
> 
> The associated *p*-value ($p < 0.05$) indicates that this relationship is **statistically significant**, meaning it is very unlikely to have arisen by chance.\
> In practical terms, this suggests that **more hydrophobic chemicals (higher $\log P$)** tend to be **more toxic** to fathead minnows (lower $\log (\text{LC}_{50})$ values).
>
> Given the strength and statistical significance of this relationship, it is reasonable to proceed to the next step and **build a linear regression model** to predict toxicity based on chemical hydrophobicity.


---

**Step 2: Building the statistical model**

Now that we have identified a strong and statistically significant linear relationship between **$\log P$** and **$\log (\text{LC}_{50})$**, we can proceed with building a **statistical model**.

In this step, we will:
- **Fit a linear regression model** with **$\log (\text{LC}_{50})$** as the dependent (response) variable and **$\log P$** as the predictor (independent variable).
- **Examine the model summary** to interpret the estimated coefficients, overall model fit, and statistical significance.
- **Analyze the residuals** to check key regression assumptions, including linearity, constant variance, and approximate normality of errors.
- **Visualize the fitted model**, including both the **confidence band** for the mean response and the **prediction band** for individual observations, on the same figure.


In [ ]:
# Fit OLS on the log scale
model_toxicity = smf.ols('log_LC50 ~ logP', data=toxicity).fit()
print(model_toxicity.summary())

> The fitted **OLS regression model** explains a substantial portion of the variability in **$\log (\text{LC}_{50})$**. The coefficient of determination is $R^2 = 0.670$, meaning that approximately **67% of the variance in toxicity** is explained by **$\log P$** alone. 
>
> The overall model is **statistically significant**, as shown by the *F*-test ($F = 85.22$, $p < 0.05$). This confirms that **$\log P$** provides meaningful explanatory power for predicting **$\log (\text{LC}_{50})$**.
>
> Interpretation of coefficients:
> - The **coefficient** for **$\log P$** is **−0.903**, and it is highly statistically significant ($p < 0.05$). This indicates a strong **negative relationship** between hydrophobicity and toxicity. As **$\log P$** increases by one unit, **$\log (\text{LC}_{50})$** decreases on average by about **0.9 units**, implying higher toxicity for more hydrophobic chemicals. (The **95% confidence interval** for the $\log P$ coefficient ranges from **−1.10 to −0.71**, and it does not include zero. This further supports the conclusion that the effect of **$\log P$** on toxicity is statistically significant.)
> - The **intercept** represents the expected value of **$\log (\text{LC}_{50})$** when **$\log P = 0$**. While this value may not have a direct physical interpretation, it is statistically significant and necessary for defining the regression line.


In [ ]:
# Residual analysis
fitted = model_toxicity.fittedvalues
resid = model_toxicity.resid

fig, axes = plt.subplots(1, 2, figsize=(8, 4))

# Residuals vs fitted
axes[0].scatter(fitted, resid, s=18, color='#fc7202')
axes[0].axhline(0, color='black', lw=1)
axes[0].set_xlabel('Fitted values ŷ')
axes[0].set_ylabel('Residuals e')
axes[0].set_title('Residuals vs Fitted (log model)')

# Q–Q plot of residuals using probplot
stats.probplot(resid, dist="norm", plot=axes[1])
axes[1].get_lines()[0].set_markerfacecolor('#fc7202')
axes[1].get_lines()[0].set_markeredgecolor('#fc7202')
axes[1].get_lines()[1].set_color('black')
axes[1].get_lines()[1].set_linestyle('--')
axes[1].set_title('Q–Q Plot of Residuals (log model)')
axes[1].set_xlabel('Theoretical quantiles')
axes[1].set_ylabel('Sample quantiles')

plt.tight_layout()
plt.show()


> As discussed earlier, for a linear regression model to be appropriate, the **residuals should behave like random noise** and be **approximately normally distributed**.\
> In the **residuals vs fitted** plot, the residuals are scattered almost randomly around zero with no clear systematic pattern. This supports the assumptions of **linearity** and **constant variance** of the errors.\
> In the **Q–Q plot**, the residuals lie close to the reference line, with only small deviations in the tails. This suggests that the assumption of **approximate normality of the residuals** is reasonably satisfied.
>
>Taken together, these diagnostics indicate that the model assumptions are met to a sufficient degree, and the fitted linear regression model can be considered **appropriate for inference and prediction** in this context.


In [ ]:
# Visualizing the toxicity model with its confidence and prediction bands
pred = model_toxicity.get_prediction()
sf = pred.summary_frame(alpha=0.05)

# Sorting by x so that the line and bands plot correctly
order = np.argsort(toxicity['logP'].values)
x_sorted = toxicity['logP'].values[order]
y_sorted = toxicity['log_LC50'].values[order]
mean_sorted = sf['mean'].values[order]
ci_low_sorted = sf['mean_ci_lower'].values[order]
ci_high_sorted = sf['mean_ci_upper'].values[order]
pi_low_sorted = sf['obs_ci_lower'].values[order]
pi_high_sorted = sf['obs_ci_upper'].values[order]

# Plotting the data, fitted line, confidence band, and prediction band
fig, ax = plt.subplots(figsize=(5, 5))

# Plotting observed data points
ax.scatter(toxicity['logP'], toxicity['log_LC50'], s=15, alpha=0.6, color='darkgray', label='Observed data')

# Plotting fitted regression line
ax.plot(x_sorted, mean_sorted, lw=2, color='#004aad', label='Fitted line (mean)')

# Adding 95% confidence band for the mean response
ax.fill_between(x_sorted, ci_low_sorted, ci_high_sorted, alpha=0.25, color='#004aad', label='95% CI (mean)')

# Adding 95% prediction band for individual observations
ax.fill_between(x_sorted, pi_low_sorted, pi_high_sorted, alpha=0.15, color='#570a6d', label='95% prediction band')

ax.set_xlabel('logP')
ax.set_ylabel('log(LC50)')
ax.legend()
plt.tight_layout()
plt.show()

#### <font color="#fc7202">Task 4: Predicting toxicity for new chemicals</font>

Now that we have built and evaluated a statistical model, the natural next step is to **use it for prediction**.

You are provided with data for **10 new chemicals** in the file `sample10.tsv`. For these chemicals, the value of **$\log P$** is known, but their toxicity has not yet been measured.

We will use the fitted model to predict their **$\text{LC}_{50}$** values.

*Reminder:* The model was developed on the **log scale** and predicts **$\log (\text{LC}_{50})$**. Therefore, after making the predictions, the results must be **transformed back to the original scale** in order to obtain $\text{LC}_{50}$ values.



In [ ]:
# Load new chemical data for prediction
new_chemicals = pd.read_csv('sample10.tsv', sep='\t')
new_chemicals

In [ ]:
# Generating predictions (log10(LC50)) for new chemicals
pred_log = model_toxicity.predict(new_chemicals['logP'])

# Adding predictions to the DataFrame
new_chemicals['pred_log_LC50'] = pred_log

# Back-transforming to the original LC50 scale (mmol/L)
new_chemicals['pred_LC50_mmol_L'] = 10 ** new_chemicals['pred_log_LC50']
new_chemicals

##### How accurate is our model?

Now that we have obtained the **predicted $\text{LC}_{50}$ values** for the 10 new chemicals, the next question is: **how accurate is our model?**  

To evaluate its predictive performance, we can compare the model’s predictions with the **experimentally measured $\text{LC}_{50}$ values** using a quantitative error metric.

One of the most widely used measures is the **Root Mean Squared Error (RMSE)**.  
RMSE quantifies the **standard deviation of the prediction errors** (residuals), indicating how far the predicted values typically deviate from the observed data points.  
Because RMSE is expressed in the **same units** as the original measurements, it is directly interpretable and widely used to assess model accuracy.  

Mathematically, for true values $y_i$ and predicted values $\hat{y}_i$ ($i = 1, \dots, n$), the RMSE is defined as: $$\text{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n} \big(\hat{y}_i - y_i\big)^2 }.$$  

To compute RMSE, we need the **true (experimental) $\text{LC}_{50}$ values**.  
For these 10 chemicals, they are provided in the file `sample10_experimental_LC50.tsv`.  

Once both the predicted and experimental values are available, the RMSE can be computed in Python using the function `root_mean_squared_error()` from `sklearn.metrics`.



In [ ]:
# Load experimental LC50 values for the new chemicals
experimental_values = pd.read_csv('sample10_experimental_LC50.tsv', sep='\t')
experimental_values

In [ ]:
# Merging experimental LC50 values with model predictions based on matching chemical name and CAS
new_chemicals = new_chemicals.merge(experimental_values, how='inner', on=['Chemical name', 'CAS'])
new_chemicals

Before calculating the **RMSE**, let’s first **visualize** the predicted and experimental $\text{LC}_{50}$ values for the 10 new chemicals.  

We’ll use the same plotting style as before: we will show the **fitted regression line**, the **confidence band** and the **prediction band**, and then overlay both the **predicted** and the **true (experimental)** values for the new chemicals.


In [ ]:
# Visualizing model fit, uncertainty bands, and the new predictions vs. experimental data
fig, ax = plt.subplots(figsize=(6, 4))

# Plotting observed data used for model fitting
ax.scatter(toxicity['logP'], toxicity['log_LC50'], s=15, alpha=0.6, color='darkgray', label='Training data')

# Plotting the fitted regression line (mean response)
ax.plot(x_sorted, mean_sorted, lw=2, color='#004aad', label='Fitted line (mean)')

# Adding 95% confidence band for the mean response
ax.fill_between(x_sorted, ci_low_sorted, ci_high_sorted, alpha=0.25, color='#004aad', label='95% CI (mean)')

# Adding 95% prediction band for individual observations
ax.fill_between(x_sorted, pi_low_sorted, pi_high_sorted, alpha=0.15, color='#570a6d', label='95% prediction band')

# Plotting predicted LC50 values (log scale) for new chemicals
ax.scatter(new_chemicals['logP'], new_chemicals['pred_log_LC50'], s=30, color='#fc7202', label='Predicted values', zorder=8)

# Plotting experimental LC50 values (log scale) for new chemicals
ax.scatter(new_chemicals['logP'], np.log10(new_chemicals['LC50 (mmol/L)']), s=30, color='#1b7173', label='Experimental values', zorder=9)

ax.set_xlabel('logP')
ax.set_ylabel('log(LC50)')
ax.legend()
plt.tight_layout()
plt.show()


> From the plot, we see that the predicted $\text{LC}_{50}$ values lie exactly on the regression line, as they are obtained directly from the model.  
> The experimental $\text{LC}_{50}$ values, however, deviate to varying degrees: depending on the chemical, some experimental values differ substantially from the model predictions, while others align more closely.

In [ ]:
# Calculating RMSE between predicted and experimental LC50 values
root_mean_squared_error(new_chemicals['LC50 (mmol/L)'], new_chemicals['pred_LC50_mmol_L'])

The RMSE is ≈ 0.237 mmol/L, which means that, on average, the model’s LC50 predictions differ from the experimental values by about 0.24 mmol/L.

##### How do we evaluate a model when no additional data are available?

In many real-world applications, we face an important practical problem: we have **only one dataset**, and after building the model we **cannot collect new data** to independently evaluate its performance.

This is actually the **most common scenario in practice**. We are given a dataset, we need to:
1. build a model,
2. assess how well it performs, and
3. then apply it to make predictions for new, unseen cases.

To do this correctly, we must plan for model evaluation **from the very beginning**.

--- 

**Holding out data for validation**

When no external validation data are available, the standard solution is to **set aside part of the original dataset** before fitting the model.

- One part of the data is used for **model fitting** (also called *training* or *calibration*).
- The remaining part is kept **completely separate** and is used **only at the end** to evaluate the model’s predictive performance (*testing*/*validation* data).

Importantly, the test data **must not be used** during model fitting and should be examined **only after the model is finalized**. (This mimics the situation of predicting truly new data.)

In addition, the **held-out test set must be representative** of the original dataset.  
This means it should cover a similar range of predictor values and reflect the same variability as the full data. If the test set contains only extreme or atypical observations, the resulting performance assessment can be misleading.

For this reason, data splitting is typically done **randomly**. In some cases, **stratified splitting** is used, where the data are split in a way that preserves the distribution of key variables (for example, ranges of toxicity or predictor values) in both the training and test sets.

--- 

**How much data should be set aside?**

The exact strategy depends on **how much data we have available**:

- **Large datasets**  
  A simple **train–test split** is often sufficient, for example:
  - 70% training / 30% testing  
  - 80% training / 20% testing  

- **Moderate-sized datasets**  
  A train–test split is still possible, but the results may depend strongly on *which* observations end up in the test set.

- **Small datasets**  
  Setting aside a large test set can waste valuable information and leave too little data for model fitting.  
  On the other hand, setting aside a **very small validation set** can also be problematic, because performance estimates become **unstable and highly sensitive** to which observations happen to be included.  

  In such cases, more efficient approaches, such as **cross-validation**, are often preferred because they make better use of the limited data while still providing a meaningful assessment of model performance.

--- 

**Common validation strategies**

Depending on data size and context, common options include:

- **Train–test split**  
  Simple and intuitive, but sensitive to how the split is done.

- **Cross-validation** (for example, *k*-fold cross-validation)  
  The data are split into several parts, and the model is trained and tested multiple times.  
  This uses data more efficiently and provides a more stable estimate of model performance.
  <p align="left">
  <img src="https://raw.githubusercontent.com/RaHub4AI/MI7032/refs/heads/main/Pictures/k_fold_cv.png"
       alt="cv"
       width="800">
</p>
 
- **Leave-one-out cross-validation (LOOCV)**  
  A special case of cross-validation, often used when datasets are very small.  
  In LOOCV, the model is trained on all observations **except one**, and the left-out observation is used for testing.  
  This process is repeated so that **each observation is used once as the test case**, and the prediction errors are then aggregated.

Each approach represents a trade-off between **bias**, **variance**, and **data efficiency**.


#### <font color="#fc7202">Task 5: Model evaluation</font>
In this example, we will look at one of the **most common ways to split data in Python**.

We will use the function `train_test_split()` from `sklearn.model_selection` to divide the original toxicity dataset, which contains **44 chemicals**, into:
- a **calibration (training) set** and
- a **validation (test) set**.

We will apply a **70:30 split**, meaning that 70% of the data are used to fit the model and the remaining 30% are held out for validation. Although this dataset is relatively small, this example clearly illustrates the standard workflow.

After splitting the data, we will:
- fit the regression model using only the calibration data,
- generate predictions for the validation set, and
- evaluate model performance by reporting the **RMSE** on the validation data.

This procedure mimics how models are typically developed and evaluated.


In [ ]:
# Split the data into calibration (70%) and validation (30%) sets
train_df, test_df = train_test_split(toxicity, test_size=0.3,random_state=42)

In [ ]:
# Fit the linear regression model on the calibration data
model_small = smf.ols('log_LC50 ~ logP', data=train_df).fit()

print(model_small.summary())

In [ ]:
# Predict log(LC50) for the validation data
log_lc50_pred = model_small.predict(test_df)

# Transform predictions and observations back to LC50 scale
lc50_pred = np.exp(log_lc50_pred)
lc50_true = np.exp(test_df['log_LC50'])

# Compute RMSE on the validation set
rmse = root_mean_squared_error(lc50_true, lc50_pred)

print(f'Validation RMSE: {rmse:.3f}')